In [1]:
import tensorflow as tf
print(tf.__version__)
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split

import unicodedata
import re
import numpy as np
import os
import io
import time

2.6.0


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
df = pd.read_csv("drive/MyDrive/Colab_data/Topic_Bifurcated_SQUAD1.csv")
df.head()

,index,question,context,text,answer_start,c_id,given_context,reference_context
0,56be85543aeaaa14008c9063,When did Beyonce start becoming popular?,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,in the late 1990s,269.0,0,"Born and raised in Houston, Texas, she perform...",Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...
1,56be85543aeaaa14008c9065,What areas did Beyonce compete in when she was...,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,singing and dancing,207.0,0,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,"Born and raised in Houston, Texas, she perform..."
2,56be85543aeaaa14008c9066,When did Beyonce leave Destiny's Child and bec...,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,2003,526.0,0,"Born and raised in Houston, Texas, she perform...",Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...
3,56bf6b0f3aeaaa14008c9601,In what city and state did Beyonce grow up?,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,"Houston, Texas",166.0,0,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,"Born and raised in Houston, Texas, she perform..."
4,56bf6b0f3aeaaa14008c9602,In which decade did Beyonce become famous?,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,late 1990s,276.0,0,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,"Born and raised in Houston, Texas, she perform..."


In [4]:
df = df[:1000]




# Load data set




In [5]:
reference_context = df["reference_context"].apply(lambda x: x).tolist()
given_context = df["given_context"].apply(lambda x: x).tolist()
target_ques = df["question"].apply(lambda x: x).tolist()

*   Clean the sentences by removing special characters.
*   Add a start and end token to each sentence.
*   Create a word index and reverse word index (dictionaries mapping from word → id and id → word).
*   Pad each sentence to a maximum length.

In [6]:
# Converts the unicode file to ascii
def unicode_to_ascii(s):
  return ''.join(c for c in unicodedata.normalize('NFD', s)
      if unicodedata.category(c) != 'Mn')


def preprocess_sentence(w):
  w = unicode_to_ascii(w.lower().strip())

  # creating a space between a word and the punctuation following it
  # eg: "he is a boy." => "he is a boy ."
  # Reference:- https://stackoverflow.com/questions/3645931/python-padding-punctuation-with-white-spaces-keeping-punctuation
  w = re.sub(r"([?.!,¿])", r" \1 ", w)
  w = re.sub(r'[" "]+', " ", w)

  # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",","¿")
  w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)

  # remove extra space
  w = w.strip()

  # adding a start and an end token to the sentence
  # so that the model know when to start and stop predicting.
  w = '<start> ' + w + ' <end>'
  return w

In [7]:
en_sentence = u"Beyonce's last record was called Lemonade"
sp_sentence = u"Beyonce was born in 1950"
print(preprocess_sentence(en_sentence))
print(preprocess_sentence(sp_sentence))
print(preprocess_sentence(sp_sentence).encode("UTF-8"))

<start> beyonce s last record was called lemonade <end>
<start> beyonce was born in <end>
b'<start> beyonce was born in <end>'


In [8]:
reference_context[:1], given_context[:1]

(['Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Their hiatus saw the release of Beyoncé\'s debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy". '],
 ["Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny's Child. Managed by her father, Mathew Knowles, the group became one of the world's best-selling girl groups of all time. "])

In [9]:
def create_dataset_squad(context_param, given_param, target_ques):
  reference_context = []
  given_context = []
  question = []
  for c,g,q in zip(context_param,given_param, target_ques):
    reference_context.append(preprocess_sentence(str(c)))
    given_context.append(preprocess_sentence(str(g)))
    question.append(preprocess_sentence(q))
  return tuple(reference_context), tuple(given_context), tuple(question)

r, g, q = create_dataset_squad(reference_context, given_context, target_ques)
print(r[-1])
print(g[-1])
print(q[-1])
print(len(r), len(q))

<start> his fellow student at the warsaw conservatory , julian fontana , had originally tried unsuccessfully to establish himself in england albert grzyma a , who in paris became a wealthy financier and society figure , often acted as chopin s adviser and gradually began to fill the role of elder brother in his life . fontana was to become , in the words of micha owski and samson , chopin s general factotum and copyist . <end>
<start> two polish friends in paris were also to play important roles in chopin s life there . <end>
<start> who is stated as a jack of all trades in service to frederic ? <end>
1000 1000


In [10]:
# Tokenize the sentence into list of words(integers) and pad the sequence to the same length
def tokenize(lang):
  lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(
      filters='')
  lang_tokenizer.fit_on_texts(lang)

  tensor = lang_tokenizer.texts_to_sequences(lang)

  tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor,
                                                         padding='post')
  return tensor, lang_tokenizer

In [11]:
def load_dataset(reference, given, target):
  # creating cleaned input, output pairs
  #targ_lang, inp_lang = create_dataset(path, num_examples)

  reference_tensor, reference_tokenizer = tokenize(reference)
  given_tensor, given_tokenizer = tokenize(given)
  target_tensor, targ_lang_tokenizer = tokenize(target)

  return reference_tensor, given_tensor, target_tensor, reference_tokenizer, given_tokenizer, targ_lang_tokenizer

In [12]:
# Try experimenting with the size of that dataset
#num_examples = 30000
# input_tensor, target_tensor, inp_lang, targ_lang = load_dataset(c,q)
reference_tensor, given_tensor, target_tensor, reference_tokenizer, given_tokenizer, targ_tokenizer = load_dataset(r, g, q)

# Calculate max_length of the target tensors
max_length_targ, max_length_reference, max_length_given = target_tensor.shape[1], reference_tensor.shape[1], given_tensor.shape[1]
print(max_length_targ, max_length_reference, max_length_given)

30 246 150


In [13]:
# train = zip(reference_tensor, given_tensor)
# train = pd.DataFrame(data=train, index=None)
# train.head()

In [14]:

# Creating training and validation sets using an 80-20 split
# input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(train, target_tensor, test_size=0.2)
reference_tensor_train, reference_tensor_val, given_tensor_train, given_tensor_val, target_tensor_train, target_tensor_val = train_test_split(reference_tensor, given_tensor, target_tensor)

# Show length
# print(len(input_tensor_train), len(target_tensor_train), len(input_tensor_val), len(target_tensor_val))
# print(input_tensor_train[1])
# print()
# print(target_tensor_train[1])

In [15]:
# input_tensor_train
# reference_tensor_train = input_tensor_train[0].values
# given_tensor_train = input_tensor_train[1].values
# 
# reference_tensor_val = input_tensor_val[0].values
# given_tensor_val = input_tensor_val[1].values

In [16]:
given_tensor_train.shape, reference_tensor_train.shape

((750, 150), (750, 246))

In [17]:
type(given_tensor_train[0])

numpy.ndarray

In [18]:
# np.expand_dims(given_tensor_train, axis=1).shape

In [19]:
# Configuration 
BUFFER_SIZE = len(reference_tensor_train)
BATCH_SIZE = 32
steps_per_epoch = len(reference_tensor_train)//BATCH_SIZE
steps_per_epoch_val = len(reference_tensor_val)//BATCH_SIZE
embedding_dim = 256  # for word embedding
units = 1024  # dimensionality of the output space of RNN
vocab_inp_size = len(given_tokenizer.word_index) + len(reference_tokenizer.word_index) + 1
vocab_tar_size = len(targ_tokenizer.word_index)+1

dataset = tf.data.Dataset.from_tensor_slices((reference_tensor_train, given_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
validation_dataset = tf.data.Dataset.from_tensor_slices((reference_tensor_val, given_tensor_val, target_tensor_val)).shuffle(BUFFER_SIZE)
validation_dataset = validation_dataset.batch(BATCH_SIZE, drop_remainder=True)

example_reference_batch, example_given_batch, example_target_batch = next(iter(dataset))
example_reference_batch.shape, example_given_batch.shape, example_target_batch.shape

(TensorShape([32, 246]), TensorShape([32, 150]), TensorShape([32, 30]))

# Basic seq2seq model: encoder and decoder

In [20]:
class Encoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
    super(Encoder, self).__init__()
    self.batch_sz = batch_sz
    self.enc_units = enc_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.enc_units,
                                   return_sequences=True,  # Whether to return the last output in the output sequence, or the full sequence. 
                                   return_state=True,  # Whether to return the last state in addition to the output.
                                   recurrent_initializer='glorot_uniform')

  def call(self, x, hidden):
    x = self.embedding(x)
    output, state = self.gru(x, initial_state = hidden)
    return output, state

  def initialize_hidden_state(self):
    return tf.zeros((self.batch_sz, self.enc_units))

In [21]:
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)

# sample input
sample_hidden = encoder.initialize_hidden_state()
sample_output, sample_hidden = encoder(example_reference_batch, sample_hidden)
print ('Encoder output shape: (batch size, sequence length, units) {}'.format(sample_output.shape))
print ('Encoder Hidden state shape: (batch size, units) {}'.format(sample_hidden.shape))

Encoder output shape: (batch size, sequence length, units) (32, 246, 1024)
Encoder Hidden state shape: (batch size, units) (32, 1024)


In [22]:
class Decoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
    super(Decoder, self).__init__()
    self.batch_sz = batch_sz
    self.dec_units = dec_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.dec_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')
    self.fc = tf.keras.layers.Dense(vocab_size)


  def call(self, x, hidden):
    # x shape after passing through embedding == (batch_size, 1, embedding_dim)
    x = self.embedding(x)

    # passing the concatenated vector to the GRU
    output, state = self.gru(x, initial_state = hidden)

    # output shape == (batch_size * 1, hidden_size)
    output = tf.reshape(output, (-1, output.shape[2]))

    # output shape == (batch_size, vocab)
    x = self.fc(output)
    return x, state

In [23]:
tf.reshape([[1,2,3],[4,5,6]], (-1, 2))

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[1, 2],
       [3, 4],
       [5, 6]], dtype=int32)>

In [24]:
decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE)

sample_decoder_output, _ = decoder(tf.random.uniform((BATCH_SIZE, 1)),
                                      sample_hidden)

print ('Decoder output shape: (batch_size, vocab size) {}'.format(sample_decoder_output.shape))

Decoder output shape: (batch_size, vocab size) (32, 1526)


In [25]:
class DotProductAttention(tf.keras.layers.Layer):
  def call(self, query, values):
    # query hidden state shape == (batch_size, hidden size)
    # query_with_time_axis shape == (batch_size, 1, hidden size)
    # values shape == (batch_size, max_len, hidden size)
    # we are doing this to broadcast addition along the time axis to calculate the score
    query_with_time_axis = tf.expand_dims(query, 1)

    # inner product, score shape == (batch_size, max_length, 1)
    score = query_with_time_axis * values
    score = tf.reduce_sum(score, axis=2)
    score = tf.expand_dims(score, 2)
    
    # attention_weights shape == (batch_size, max_length, 1)
    attention_weights = tf.nn.softmax(score, axis=1)

    # context_vector shape after sum == (batch_size, hidden_size)
    context_vector = attention_weights * values
    context_vector = tf.reduce_sum(context_vector, axis=1)

    return context_vector, attention_weights

In [26]:
attention_layer = DotProductAttention()
attention_result, attention_weights = attention_layer(sample_hidden, sample_output)

print("Attention result shape: (batch size, units) {}".format(attention_result.shape))
print("Attention weights shape: (batch_size, sequence_length, 1) {}".format(attention_weights.shape))

Attention result shape: (batch size, units) (32, 1024)
Attention weights shape: (batch_size, sequence_length, 1) (32, 246, 1)


In [27]:
class BahdanauAttention(tf.keras.layers.Layer):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.W1 = tf.keras.layers.Dense(units)
    self.W2 = tf.keras.layers.Dense(units)
    self.V = tf.keras.layers.Dense(1)

  def call(self, query, values):
    # query hidden state shape == (batch_size, hidden size)
    # query_with_time_axis shape == (batch_size, 1, hidden size)
    # values shape == (batch_size, max_len, hidden size)
    query_with_time_axis = tf.expand_dims(query, 1)
    # values = query_with_time_axis
    # print(values.shape)
    # print(query_with_time_axis.shape)
    # score shape == (batch_size, max_length, 1)
    # we get 1 at the last axis because we are applying score to self.V
    # the shape of the tensor before applying self.V is (batch_size, max_length, units)
    score = self.V(tf.nn.tanh(
        self.W1(values) + self.W2(query_with_time_axis)))

    # attention_weights shape == (batch_size, max_length, 1)
    attention_weights = tf.nn.softmax(score, axis=1)

    # context_vector shape after sum == (batch_size, hidden_size)
    context_vector = attention_weights * values
    context_vector = tf.reduce_sum(context_vector, axis=1)

    return context_vector, attention_weights

In [28]:
class DecoderWithAttention(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz, attention_layer = None):
    super(DecoderWithAttention, self).__init__()
    self.batch_sz = batch_sz
    self.dec_units = dec_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.dec_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')
    self.fc = tf.keras.layers.Dense(vocab_size)

    # used for attention
    self.attention = attention_layer

  def call(self, x, hidden, enc_output):
    # x shape after passing through embedding == (batch_size, 1, embedding_dim)
    x = self.embedding(x)
    
    attention_weights1 = None
    
    if self.attention1:
      # enc_output shape == (batch_size, max_length, hidden_size)
      context_vector1, attention_weights1 = self.attention1(hidden, enc_output)
      # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
      # print(x.shape)
      x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)
      

    attention_weights2 = None
    
    if self.attention2:
      # enc_output shape == (batch_size, max_length, hidden_size)
      context_vector2, attention_weights2 = self.attention2(enc_hidden, enc_output)
      # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
      x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)
    
    # passing the concatenated vector to the GRU
    output, state = self.gru(x, initial_state = hidden)

    # output shape == (batch_size * 1, hidden_size)
    output = tf.reshape(output, (-1, output.shape[2]))

    # output shape == (batch_size, vocab)
    x = self.fc(output)

    return x, state, attention_weights

In [37]:
class BahdanauAttention(tf.keras.layers.Layer):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.W1 = tf.keras.layers.Dense(units)
    self.W2 = tf.keras.layers.Dense(units)
    self.V = tf.keras.layers.Dense(1)

  def call(self, query, values):
    # query hidden state shape == (batch_size, hidden size)
    # query_with_time_axis shape == (batch_size, 1, hidden size)
    # values shape == (batch_size, max_len, hidden size)
    query_with_time_axis = tf.expand_dims(query, 1)
    values = query_with_time_axis
    # print("values.shape", values.shape)
    # print("query_with_time_axis.shape", query_with_time_axis.shape)

    # (64, 358, 1024) ->values.shape
    # (64, 1, 1024) -> query_with_time_axis.shape

    # score shape == (batch_size, max_length, 1)
    # we get 1 at the last axis because we are applying score to self.V
    # the shape of the tensor before applying self.V is (batch_size, max_length, units)
    score = self.V(tf.nn.tanh(
        self.W1(values) + self.W2(query_with_time_axis)))
    # print("self.W1(values).shape ->", self.W1(values).shape)
    # print("self.W2(query_with_time_axis).shape" , self.W2(query_with_time_axis).shape)
    # print("score.shape", score.shape)
    # (64, 358, 1024) -> self.W1(values).shape
    # (64, 1, 1024) -> self.W2(query_with_time_axis).shape
    # (64, 358, 1) -> score.shape

    # attention_weights shape == (batch_size, max_length, 1)
    attention_weights = tf.nn.softmax(score, axis=1)

    # context_vector shape after sum == (batch_size, hidden_size)
    context_vector = attention_weights * values
    # print(context_vector.shape)
    # (64, 358, 1024)

    context_vector = tf.reduce_sum(context_vector, axis=1)
    # print(context_vector.shape)
    # (64, 1024)
    return context_vector, attention_weights

In [30]:
class DecoderWithDualAttention(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz, attention_layer1 = None, attention_layer2 = None):
    super(DecoderWithDualAttention, self).__init__()
    self.batch_sz = batch_sz
    self.dec_units = dec_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.dec_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')
    self.fc = tf.keras.layers.Dense(vocab_size)

    # used for attention
    self.attention1 = attention_layer1
    self.attention2 = attention_layer2

  def call(self, x, hidden1, hidden2, enc_output1, enc_output2):
    # x shape after passing through embedding == (batch_size, 1, embedding_dim)
    x = self.embedding(x)
    y = x
    attention_weights1 = None   
    
    if self.attention1:
      # enc_output shape == (batch_size, max_length, hidden_size)
      context_vector1, attention_weights1 = self.attention1(hidden1, enc_output1)
      # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
      # print(x.shape)
      x = tf.concat([tf.expand_dims(context_vector1, 1), x], axis=-1)
      # print(x.shape)
    
    attention_weights2 = None
    
    if self.attention2:
      # enc_output shape == (batch_size, max_length, hidden_size)
      context_vector2, attention_weights2 = self.attention2(hidden2, enc_output2)
      # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
      y = tf.concat([tf.expand_dims(context_vector2, 1), y], axis=-1)


    # passing the concatenated vector to the GRU
    output1, state1 = self.gru(x, initial_state = hidden1)
    output2, state2 = self.gru(y, initial_state = hidden2)

    # output shape == (batch_size * 1, hidden_size)
    output1 = tf.reshape(output1, (-1, output1.shape[2]))
    output2 = tf.reshape(output2, (-1, output2.shape[2]))

    output = tf.concat([output1, output2], axis = 1)

    # output shape == (batch_size, vocab)
    final_output = self.fc(output)

    return final_output, state1, state2, attention_weights1, attention_weights2

In [31]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0)) 
  loss_ = loss_object(real, pred)
  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask
  return tf.reduce_mean(loss_)

In [32]:
print(loss_object([1,2],[[0,0.6,0.3,0.1],[0,0.6,0.3,0.1]]))
print(loss_function([1,2],[[0,0.6,0.3,0.1],[0,0.6,0.3,0.1]]))

tf.Tensor([1.063386  1.3633859], shape=(2,), dtype=float32)
tf.Tensor(1.2133859, shape=(), dtype=float32)


In [33]:
optimizer = tf.keras.optimizers.Adam()

def get_train_step_func():

  @tf.function
  def train_step(inp, targ, aux, enc_hidden1, enc_hidden2, encoder1, encoder2, decoder):
    loss = 0

    with tf.GradientTape() as tape: # for automatic differentiation
      enc_output1, enc_hidden1 = encoder1(inp, enc_hidden1)
      enc_output2, enc_hidden2 = encoder2(aux, enc_hidden2)

      dec_hidden1 = enc_hidden1
      dec_hidden2 = enc_hidden2
      # enc_hidden -> 

      # Check whether we should concatenate dec_hidden and dec_hidden2 or keep it separate
      # dec_hidden2 = enc_hidden2

      dec_input = tf.expand_dims([targ_tokenizer.word_index['<start>']] * BATCH_SIZE, 1)

      # Teacher forcing - feeding the target as the next input
      for t in range(1, targ.shape[1]):
        # passing enc_output to the decoder
        predictions, dec_hidden1, dec_hidden2, _, _ = decoder(dec_input, dec_hidden1, dec_hidden2, enc_output1, enc_output2)
        # x, y, hidden1, hidden2, enc_output1, enc_output2

        loss += loss_function(targ[:, t], predictions)

        # using teacher forcing
        dec_input = tf.expand_dims(targ[:, t], 1)

    batch_loss = (loss / int(targ.shape[1]))

    variables = encoder1.trainable_variables + encoder2.trainable_variables + decoder.trainable_variables

    gradients = tape.gradient(loss, variables)

    optimizer.apply_gradients(zip(gradients, variables))

    return batch_loss
    
  return train_step

In [34]:
def caculate_validation_loss(inp, targ, aux, enc_hidden1, enc_hidden2, encoder1, encoder2, decoder):
  loss = 0
  enc_output1, enc_hidden1 = encoder1(inp, enc_hidden1)
  enc_output2, enc_hidden2 = encoder2(aux, enc_hidden1)
  dec_hidden1 = enc_hidden1
  dec_hidden2 = enc_hidden2
  dec_input = tf.expand_dims([targ_tokenizer.word_index['<start>']] * BATCH_SIZE, 1)

  # Teacher forcing - feeding the target as the next input
  # for t in range(1, targ.shape[1]):
  #   predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)
  #   loss += loss_function(targ[:, t], predictions)
  #   dec_input = tf.expand_dims(targ[:, t], 1)

  # loss = loss / int(targ.shape[1])
  # return loss

  for t in range(1, targ.shape[1]):
        # passing enc_output to the decoder
        predictions, dec_hidden1, dec_hidden2, _, _ = decoder(dec_input, dec_hidden1, dec_hidden2, enc_output1, enc_output2)
        # x, y, hidden1, hidden2, enc_output1, enc_output2

        loss += loss_function(targ[:, t], predictions)

        # using teacher forcing
        dec_input = tf.expand_dims(targ[:, t], 1)

  loss = loss / int(targ.shape[1])
  return loss

In [35]:
def training_seq2seq(epochs, attention1, attention2):
  encoder1 = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)
  encoder2 = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)
  decoder = DecoderWithDualAttention(vocab_tar_size, embedding_dim, units, BATCH_SIZE, attention1, attention2)
  train_step_func = get_train_step_func()
  training_loss = []
  validation_loss = []

  for epoch in range(epochs):
    print()
    print(epoch)
    start = time.time()
    enc_hidden1 = encoder1.initialize_hidden_state()
    enc_hidden2 = encoder2.initialize_hidden_state()
    total_loss = 0

    for (batch, (aux, inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
      print(batch)
      batch_loss = train_step_func(inp, targ, aux, enc_hidden1, enc_hidden2, encoder1, encoder2, decoder)
      # inp, targ, aux, enc_hidden, enc_hidden2, encoder, encoder2, decoder
      total_loss += batch_loss

      if batch % 100 == 0:
        print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1, batch, batch_loss))
        
    enc_hidden1 = encoder1.initialize_hidden_state()
    enc_hidden2 = encoder2.initialize_hidden_state()
    total_val_loss = 0
    for (batch, (aux, inp, targ)) in enumerate(validation_dataset.take(steps_per_epoch)):
      val_loss = caculate_validation_loss(inp, targ, aux, enc_hidden1, enc_hidden2, encoder1, encoder2, decoder)
      total_val_loss += val_loss

    training_loss.append(total_loss / steps_per_epoch)
    validation_loss.append(total_val_loss / steps_per_epoch_val)
    print('Epoch {} Loss {:.4f} Validation Loss {:.4f}'.format(epoch + 1,
                                        training_loss[-1], validation_loss[-1]))
    print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))
  return encoder1, encoder2, decoder, training_loss, validation_loss

## Training seq2seq without attention



In [ ]:
epochs = 10
attention = None

print("Running seq2seq model without attention")
encoder, decoder, training_loss, validation_loss = training_seq2seq(epochs, attention)

tloss = training_loss
vloss = validation_loss

Running seq2seq model without attention


TypeError: ignored

## Training seq2seq with dot product attention

In [ ]:
attention = DotProductAttention()
print("Running seq2seq model with dot product attention")
encoder_dp, decoder_dp, training_loss, validation_loss = training_seq2seq(epochs, attention)

tloss = np.vstack((tloss, training_loss))
vloss = np.vstack((vloss, validation_loss))

Running seq2seq model with dot product attention
Epoch 1 Batch 0 Loss 4.6264
Epoch 1 Batch 100 Loss 2.4235
Epoch 1 Batch 200 Loss 2.1550
Epoch 1 Batch 300 Loss 1.8926
Epoch 1 Loss 2.4485 Validation Loss 1.9181
Time taken for 1 epoch 42.6515851020813 sec

Epoch 2 Batch 0 Loss 1.8629
Epoch 2 Batch 100 Loss 1.7040
Epoch 2 Batch 200 Loss 1.4752
Epoch 2 Batch 300 Loss 1.5064
Epoch 2 Loss 1.6576 Validation Loss 1.5580
Time taken for 1 epoch 34.465418100357056 sec

Epoch 3 Batch 0 Loss 1.3883
Epoch 3 Batch 100 Loss 1.3110
Epoch 3 Batch 200 Loss 1.4096
Epoch 3 Batch 300 Loss 1.3529
Epoch 3 Loss 1.3195 Validation Loss 1.3515
Time taken for 1 epoch 34.06086468696594 sec

Epoch 4 Batch 0 Loss 1.1593
Epoch 4 Batch 100 Loss 1.0606
Epoch 4 Batch 200 Loss 0.9367
Epoch 4 Batch 300 Loss 0.9321
Epoch 4 Loss 1.0839 Validation Loss 1.2153
Time taken for 1 epoch 34.23249864578247 sec

Epoch 5 Batch 0 Loss 0.8570
Epoch 5 Batch 100 Loss 0.9223
Epoch 5 Batch 200 Loss 0.9343
Epoch 5 Batch 300 Loss 0.7972
Epoch

## Training seq2seq with Bahdanau attention

In [38]:
epochs = 1

attention1 = BahdanauAttention(units)
attention2 = BahdanauAttention(units)
print("Running seq2seq model with Bahdanau attention")
encoder1_bah, encoder_bah2, decoder_bah, training_loss, validation_loss = training_seq2seq(epochs, attention1, attention2)

# tloss = np.vstack((tloss, training_loss))
# vloss = np.vstack((vloss, validation_loss))

Running seq2seq model with Bahdanau attention

0
0
Epoch 1 Batch 0 Loss 3.0160
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
Epoch 1 Loss 2.4843 Validation Loss 2.2662
Time taken for 1 epoch 71.1959776878357 sec



In [41]:
encoder1_bah.save("./encoder1")

INFO:tensorflow:Assets written to: ./encoder1/assets


INFO:tensorflow:Assets written to: ./encoder1/assets


In [42]:
encoder_check = tf.keras.models.load_model("./encoder1")

In [45]:
decoder_bah.save("./decoder")

INFO:tensorflow:Assets written to: ./decoder/assets


INFO:tensorflow:Assets written to: ./decoder/assets


In [46]:
decoder_check = tf.keras.models.load_model("./decoder")

In [54]:
# decoder_check.get_weights() == decoder_bah.get_weights()

In [55]:
encoder_bah2.save("./encoder2")
encoder_check2 = tf.keras.models.load_model("./encoder2")

INFO:tensorflow:Assets written to: ./encoder2/assets


INFO:tensorflow:Assets written to: ./encoder2/assets


In [68]:
encoder_check = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)
encoder_check2 = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)
decoder_check = DecoderWithDualAttention(vocab_tar_size, embedding_dim, units, BATCH_SIZE, attention1, attention2)
# encoder_check = tf.keras.models.load_model("./encoder1")
# encoder_check2 = tf.keras.models.load_model("./encoder2")
# decoder_check = tf.keras.models.load_model("./decoder")
encoder_check.load_weights("./encoder1")
encoder_check2.load_weights("./encoder2")
decoder_check.load_weights("./decoder")

In [56]:
def translate(sentence, encoder, decoder):
  attention_plot = np.zeros((max_length_targ, max_length_inp))

  sentence = preprocess_sentence(sentence)

  inputs = [inp_lang.word_index[i] for i in sentence.split(' ')]
  inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
                                                         maxlen=max_length_inp,
                                                         padding='post')
  inputs = tf.convert_to_tensor(inputs)

  result = ''

  hidden = [tf.zeros((1, units))]
  enc_out, enc_hidden = encoder(inputs, hidden)

  dec_hidden = enc_hidden
  dec_input = tf.expand_dims([targ_tokenizer.word_index['<start>']], 0)

  for t in range(max_length_targ):
    predictions, dec_hidden, attention_weights = decoder(dec_input,
                                                         dec_hidden,
                                                         enc_out)

    predicted_id = tf.argmax(predictions[0]).numpy()

    result += targ_tokenizer.index_word[predicted_id] + ' '

    # until the predicted word is <end>.
    if targ_tokenizer.index_word[predicted_id] == '<end>':
      return result, sentence

    # the predicted ID is fed back into the model, no teacher forcing.
    dec_input = tf.expand_dims([predicted_id], 0)

  return result, sentence

In [60]:
def translate(g_context, r_context, encoder1, encoder2, decoder):
  attention_plot1 = np.zeros((max_length_targ, max_length_given))
  attention_plot2 = np.zeros((max_length_targ, max_length_reference))

  g_context = preprocess_sentence(g_context)

  given = [given_tokenizer.word_index[i] for i in g_context.split(' ')]
  given = tf.keras.preprocessing.sequence.pad_sequences([given],
                                                         maxlen=max_length_given,
                                                         padding='post')
  given = tf.convert_to_tensor(given)

  
  r_context = preprocess_sentence(r_context)
  
  ref = [reference_tokenizer.word_index[i] for i in r_context.split(' ')]
  ref = tf.keras.preprocessing.sequence.pad_sequences([ref],
                                                         maxlen=max_length_reference,
                                                         padding='post')
  ref = tf.convert_to_tensor(ref)


  result = ''

  hidden = [tf.zeros((1, units))]
  enc_out1, enc_hidden1 = encoder1(given, hidden)
  enc_out2, enc_hidden2 = encoder2(ref, hidden)

  dec_hidden1 = enc_hidden1
  dec_hidden2 = enc_hidden2
  dec_input = tf.expand_dims([targ_tokenizer.word_index['<start>']], 0)

  for t in range(max_length_targ):
    # predictions, dec_hidden, attention_weights = decoder(dec_input,
    #                                                      dec_hidden,
    #                                                      enc_out)

    predictions, dec_hidden1, dec_hidden2, _, _ = decoder(dec_input, dec_hidden1, dec_hidden2, enc_out1, enc_out2)

    predicted_id = tf.argmax(predictions[0]).numpy()

    result += targ_tokenizer.index_word[predicted_id] + ' '

    # until the predicted word is <end>.
    if targ_tokenizer.index_word[predicted_id] == '<end>':
      return result

    # the predicted ID is fed back into the model, no teacher forcing.
    dec_input = tf.expand_dims([predicted_id], 0)

  return result

In [63]:
translate("is my country. ","Its a really and beautiful place to live. ", encoder1_bah, encoder_bah2, decoder_bah)

'what did did did beyonce beyonce beyonce ? ? ? ? ? ? <end> '

In [71]:
import warnings
warnings.simplefilter("ignore")
translate("is my country. ","Its a really and beautiful place to live. ", encoder_check, encoder_check2, decoder_check)

'what did did did beyonce beyonce beyonce ? ? ? ? ? ? <end> '

In [70]:
# attention = BahdanauAttention(units)
# encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)
# decoder = DecoderWithAttention(vocab_tar_size, embedding_dim, units, BATCH_SIZE, attention)
# loss = 0
# with tf.GradientTape() as tape: # for automatic differentiation
#     sample_hidden = encoder.initialize_hidden_state()
#     enc_output, enc_hidden = encoder(example_input_batch, sample_hidden)

#     dec_hidden = enc_hidden
#     dec_input = tf.expand_dims([targ_lang.word_index['<start>']] * BATCH_SIZE, 1)
#     # Teacher forcing - feeding the target as the next input
#     for t in range(1, example_target_batch.shape[1]):
#         # passing enc_output to the decoder
#         predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)
#         loss += loss_function(example_target_batch[:, t], predictions)
#         # using teacher forcing
#         dec_input = tf.expand_dims(example_target_batch[:, t], 1)